In [1]:
# imports
from da import defer_acceptance
from copy import deepcopy

In [2]:
'''
Before Reduction:
    Pf1 : w1w2, w1w5, w2w5, w1w3, w4w5, w2w4, w1w4, w3w4, w3w5, w2w3, w1, w4, w3, w2, w5
    Pf2 : w3w6, w3w5, w5w6, w2w5, w1w3, w2w6, w1w5, w1w2, w2w3, w1w6, w1, w2, w3, w5, w6
    Pf3 : w2w4, w1w2, w3w4, w2w3, w1w3, w1w4, w1, w2, w3, w4
    Pw1 : f3, f1, f2
    Pw2 : f2 f3, f1 f3, f1 f2, f1, f2, f3
    Pw3 : f1, f2
    Pw4 : f1, f3, f2
    Pw5 : f2, f3
    Pw6 : f1 f3, f3, f1

After Reduction:
    PμFf1: w1w2, w1w3, w2w4, w1w4,w3w4, w2w3, w1, w4, w3, w2
    PμFf2: w3w5,w2w5, w2w3, w2, w3, w5
    PμFf3: w2w4,w1w2, w1w4, w1, w2, w3, w4
    PμFw1 : f3, f1
    PμFw2 : f2 f3, f1 f3, f1 f2, f1, f2, f3
    PμFw3 : f1, f2
    PμFw4 : f1, f3
    PμFw5 : f2
    PμFw6 : Æ
'''

'\nBefore Reduction:\n    Pf1 : w1w2, w1w5, w2w5, w1w3, w4w5, w2w4, w1w4, w3w4, w3w5, w2w3, w1, w4, w3, w2, w5\n    Pf2 : w3w6, w3w5, w5w6, w2w5, w1w3, w2w6, w1w5, w1w2, w2w3, w1w6, w1, w2, w3, w5, w6\n    Pf3 : w2w4, w1w2, w3w4, w2w3, w1w3, w1w4, w1, w2, w3, w4\n    Pw1 : f3, f1, f2\n    Pw2 : f2 f3, f1 f3, f1 f2, f1, f2, f3\n    Pw3 : f1, f2\n    Pw4 : f1, f3, f2\n    Pw5 : f2, f3\n    Pw6 : f1 f3, f3, f1\n\nAfter Reduction:\n    PμFf1: w1w2, w1w3, w2w4, w1w4,w3w4, w2w3, w1, w4, w3, w2\n    PμFf2: w3w5,w2w5, w2w3, w2, w3, w5\n    PμFf3: w2w4,w1w2, w1w4, w1, w2, w3, w4\n    PμFw1 : f3, f1\n    PμFw2 : f2 f3, f1 f3, f1 f2, f1, f2, f3\n    PμFw3 : f1, f2\n    PμFw4 : f1, f3\n    PμFw5 : f2\n    PμFw6 : Æ\n'

In [3]:
'''
Reduction Step 1:
    In Step 1 (a), for each
    f ∈ F, if a worker is not in μ( f ) but belongs to a subset that is Blair-preferred to
    μ( f ), the procedure eliminates each subset that contains this worker from firm f ’s list
    of acceptable subsets. Step 1 (b) performs an analogous elimination in each worker’s
    preference list.
'''
# Before step 1: Pf2 : w3w6, w3w5, w5w6, w2w5, w1w3, w2w6, w1w5, w1w2, w2w3, w1w6, w1, w2, w3, w5, w6
# After step 1: Pf2 : w3w5, w2w5, w1w3, w1w5, w1w2, w2w3, w1, w2, w3, w5
# Remove: w3w6, w5w6, w2w6, w1w6, w6 - All subset including w6

'\nReduction Step 1:\n    In Step 1 (a), for each\n    f ∈ F, if a worker is not in μ( f ) but belongs to a subset that is Blair-preferred to\n    μ( f ), the procedure eliminates each subset that contains this worker from firm f ’s list\n    of acceptable subsets. Step 1 (b) performs an analogous elimination in each worker’s\n    preference list.\n'

In [82]:
# Return subsets assigned in the optimal stable matchings
def get_u(u1, u2):
    u_temp = [u1, u2]
    u = {}
    for k in u2.keys():
        u[k] = tuple(u[k] for u in u_temp)
    return u

# return dictionary where value is a set(list)
def get_u_sets(u):
    u_sets = {}
    for k in u.keys():
        lst = list(u[k])
        u_sets[k] = list(set(lst[0] + lst[1]))
    return u_sets

# sets --> 2-d array(value of preference dictionary); set1/set2 --> 1-d array
def blair_preferred(sets, set1, set2):
    if sets.index(set1) < sets.index(set2):
        return True
    return False

# u_sets --> array(value of u_sets); sets --> array
def set_in_u(u_set, sets):
    for s in sets:
        if s not in u_set:
            return True
    return False

def reduction_step1_step2(firms_prefs, workers_prefs):

    uf1, uw1 = defer_acceptance(males_prefs=deepcopy(firms_prefs), females_prefs=deepcopy(workers_prefs)) # DA - firms proposing
    uw2, uf2 = defer_acceptance(males_prefs=deepcopy(workers_prefs), females_prefs=deepcopy(firms_prefs)) # DA - workers proposing
    uf = get_u(uf1, uf2)
    uw = get_u(uw2, uw1)
    uf_sets = get_u_sets(uf)
    uw_sets = get_u_sets(uw)

    # Step 1 and Step 2 of reduction for one side
    # Output: preference list after reduction
    def reduction_for_oneside(preference_lists, u_sets, u1, u2):
        delete = {}
        for keys, values in preference_lists.items():
            for value in values:
                for v in value:
                    if keys in u_sets:
                        # Step 1:
                        if v not in u_sets[keys] and blair_preferred(preference_lists[keys], value, u1[keys]):
                            if keys not in delete:
                                delete[keys] = [v]
                            else:
                                if v not in delete[keys]:
                                    delete[keys].append(v)

                        # Step 2:
                        elif v not in u_sets[keys] and blair_preferred(preference_lists[keys], u2[keys], value):
                            if keys not in delete:
                                delete[keys] = [v]
                            else:
                                if v not in delete[keys]:
                                    delete[keys].append(v)

        return delete
    # Reduction on firms' side:
    delete_workers = reduction_for_oneside(firms_prefs, uf_sets, uf1, uf2)
    # Reduction on workers' side:
    delete_firms = reduction_for_oneside(workers_prefs, uw_sets, uw2, uw1)

    def delete_sets(preference_lists, delete):
        new_preference_lists = deepcopy(preference_lists)
        for keys, values in preference_lists.items():
            
            for value in values:
                if keys in delete:
                    for v in delete[keys]:
                        if v in value and value in new_preference_lists[keys]:
                            new_preference_lists[keys].remove(value)
        return new_preference_lists
    
    new_firms = delete_sets(firms_prefs, delete_workers)
    new_workers = delete_sets(workers_prefs, delete_firms)

    return [new_firms, new_workers]

def reduction(firms_prefs, workers_prefs):
    # Perform reduction step 1 and step 2
    new_firms, new_workers = reduction_step1_step2(firms_prefs, workers_prefs)




In [83]:
# E.g.1 - Paper's Eg

print("E.g.1 - Paper's Eg: \n")

firms_prefs = {
                        1: [[1,2],[1,5],[2,5],[1,3],[4,5],[2,4],[1,4],[3,4],[3,5],[2,3],[1],[4],[3],[2],[5]],
                        2: [[3,6],[3,5],[5,6],[2,5],[1,3],[2,6],[1,5],[1,2],[2,3],[1,6],[1],[2],[3],[5],[6]],
                        3: [[2,4],[1,2],[3,4],[2,3],[1,3],[1,4],[1],[2],[3],[4]]
              }

workers_prefs = {
                        1: [[3],[1],[2]],
                        2: [[2,3],[1,3],[1,2],[1],[2],[3]],
                        3: [[1],[2]],
                        4: [[1],[3],[2]],
                        5: [[2],[3]],
                        6: [[1,3],[3],[1]]
                }

reduction_step1_step2(firms_prefs, workers_prefs)



E.g.1 - Paper's Eg: 



[{1: [[1, 2], [1, 3], [2, 4], [1, 4], [3, 4], [2, 3], [1], [4], [3], [2]],
  2: [[3, 5], [2, 5], [2, 3], [2], [3], [5]],
  3: [[2, 4], [1, 2], [1, 4], [1], [2], [4]]},
 {1: [[3], [1]],
  2: [[2, 3], [1, 3], [1, 2], [1], [2], [3]],
  3: [[1], [2]],
  4: [[1], [3]],
  5: [[2]],
  6: [[1, 3], [3], [1]]}]